In [ ]:
%load_ext autoreload
%autoreload 2

import math

import matplotlib.pyplot as plt

from penai.hierarchy_generation.inference import HierarchyInferencer
from penai.hierarchy_generation.vis import (
    InteractiveHTMLHierarchyVisualizer,
    InteractiveSVGHierarchyVisualizer,
)
from penai.llm.llm_model import RegisteredLLM
from penai.registries.projects import SavedPenpotProject
from penai.registries.web_drivers import RegisteredWebDriver
from penai.utils.io import ResultWriter
from penai.utils.vis import ShapeVisualizer

plt.rcParams["figure.figsize"] = (20, 40)

In [ ]:
page_svg = SavedPenpotProject.INTERACTIVE_MUSIC_APP.load_page_svg_with_viewboxes(
    "Interactive music app",
)
playlist_shape = page_svg.get_shape_by_name("Playlist")

In [ ]:
visualizer = ShapeVisualizer(RegisteredWebDriver.CHROME)
visualizations = list(visualizer.visualize_bboxes_in_shape(playlist_shape))

In [ ]:
# plot_image_grid([vis.image for vis in visualizations])

In [ ]:
hierarchy_inference = HierarchyInferencer(model=RegisteredLLM.GPT4O)
hierarchy, visualizations = hierarchy_inference.infer_shape_hierarchy(
    playlist_shape,
    return_visualizations=True,
)

In [ ]:
visualizations = list(ShapeVisualizer().visualize_bboxes_in_shape(playlist_shape))

img_per_row = 4

fig, axes = plt.subplots(math.ceil(len(visualizations) / img_per_row), img_per_row)

for i, ax in enumerate(axes.flat):
    if i < len(visualizations):
        ax.imshow(visualizations[i].image)
    ax.axis("off")

plt.show()

In [ ]:
result_writer = ResultWriter(".")

hierarchy_svg_visualizer = InteractiveSVGHierarchyVisualizer(hierarchy, playlist_shape)
svg_filename = "hierarchy.svg"
hierarchy_svg_visualizer.write_svg(result_writer.path(svg_filename))

hierarchy_html_visualizer = InteractiveHTMLHierarchyVisualizer(svg_filename, hierarchy)
hierarchy_html_visualizer.write_html(result_writer.path("hierarchy.html"))